### Basic Nerual Network
Followed tutioral from:
https://www.youtube.com/watch?v=VMj-3S1tku0&t=2143s&ab_channel=AndrejKarpathy

In [63]:
#Libraries to be install
import numpy as np
import math
import graphviz as gv
from graphviz import Digraph
import random

# Nerual Network Math #
Derivative: $(f(x+h) - f(x)) \over h$ \
$ tanh (x) =  \frac{e^x-e^{-x}}{e^x+e^{-x}} $ 

(Neuron - Neuron) ----- Edge Weight Neuron ------ (Output Neuron)


In [102]:
# <---prev---[Neuron: Data, Grad]---next--->

class Neuron:
    def __init__(self, value, _children=()):
        self.value = value
        self.grad = 0.0
        self.next = None
        self._prev = set(_children)
        self.children = []

In [106]:
from graphviz import Digraph

def trace(Neuron):
    

def draw(setNeuron):
    # Create a new directed graph
    g = Digraph(format='svg', graph_attr={'rankdir': 'LR'})

    # Add nodes to the graph
    # for any value in the graph, create a rectangular ('record') node for it

    list = []
    for n in setNeuron:
        uid = str(id(n))
        list.append(uid)
        g.node(uid,label = "{ data %.4f | grad %.4f }" % (n.value, n.grad), shape='record')
   
    # # Add edges to the graph
    g.edge(list[0], list[1])
    g.edge(list[1], list[2])

    # Save and render the graph to a file
    return g

a = Neuron(1)
b = Neuron(2)
c = Neuron(3)
a._prev = (b)
b._prev = (c)

lstInput = [a]
draw(lstInput)


AttributeError: 'list' object has no attribute '_prev'

In [ ]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  # Returns string representation of the object
  def __repr__(self):
    return f"Value(data={self.data})"
  
  # Returns addition of other Value object
  def __add__(self, other):
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  # Returns multiplcation of other Value object 
  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  # Implmentation of hyperbolic tangent
  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  # Back propogation for nodes
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [65]:
root = Neuron('Root')
child1 = Neuron('Child 1')
child2 = Neuron('Child 2')

root.add_child(child1)
root.add_child(child2)

for x in root.children:
    print(x.value)

Child 1
Child 2


In [66]:
def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [72]:
draw_dot(root)

AttributeError: 'Neuron' object has no attribute '_prev'

# Expresion #
$a^{(1)}=\sigma (Wa^{(0)}+b)$

In [69]:
A = np.array([[2, 4], [5, -6]])
B = np.array([[9, -3], [3, 6]])